# 1. DATABASE OLUŞTURMA 

In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torchvision
import sklearn
from  sklearn.model_selection import train_test_split 
import os 
import cv2

In [17]:
path= "E:\\datasets\\defect_detection\\"

In [18]:
df=pd.read_csv(path+"jarlids_annots.csv")

In [19]:
df

,Unnamed: 0,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,0,p1.JPG,167996,{},12,0,"{""name"":""rect"",""x"":70,""y"":46,""width"":120,""heig...","{""type"":""intact""}"
1,1,p1.JPG,167996,{},12,1,"{""name"":""rect"",""x"":195,""y"":60,""width"":118,""hei...","{""type"":""intact""}"
2,2,p1.JPG,167996,{},12,2,"{""name"":""rect"",""x"":318,""y"":25,""width"":144,""hei...","{""type"":""damaged""}"
3,3,p1.JPG,167996,{},12,3,"{""name"":""rect"",""x"":474,""y"":13,""width"":135,""hei...","{""type"":""intact""}"
4,4,p1.JPG,167996,{},12,4,"{""name"":""rect"",""x"":45,""y"":162,""width"":123,""hei...","{""type"":""intact""}"
...,...,...,...,...,...,...,...,...
1854,1854,t1.JPG,174414,{},6,1,"{""name"":""rect"",""x"":232,""y"":34,""width"":147,""hei...","{""type"":""intact""}"
1855,1855,t1.JPG,174414,{},6,2,"{""name"":""rect"",""x"":451,""y"":89,""width"":124,""hei...","{""type"":""intact""}"
1856,1856,t1.JPG,174414,{},6,3,"{""name"":""rect"",""x"":80,""y"":248,""width"":138,""hei...","{""type"":""damaged""}"
1857,1857,t1.JPG,174414,{},6,4,"{""name"":""rect"",""x"":244,""y"":284,""width"":146,""he...","{""type"":""intact""}"


In [21]:
import sqlite3
import os
import pandas as pd
from faker import Faker
import json

fake = Faker()

conn = sqlite3.connect('jar_lids_db_v3.sqlite')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        image_path TEXT,
        production_date DATE,
        xmin INTEGER,  -- xmin sütunu
        ymin INTEGER,  -- ymin sütunu
        width INTEGER,  -- width sütunu
        height INTEGER  -- height sütunu
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS quality_control (
        qc_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        image_path TEXT,
        qc_date DATE,
        quality_grade INTEGER,
        notes TEXT,
        xmin INTEGER,  -- xmin sütunu
        ymin INTEGER,  -- ymin sütunu
        width INTEGER,  -- width sütunu
        height INTEGER,  -- height sütunu
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
''')
 
image_folder = "E:\\datasets\\defect_detection" 

# Ürünleri ekleme
for idx, row in df.iterrows():
    filename = row['filename']
    image_path = os.path.join(image_folder, filename)
    
    # Üretim tarihi (rastgele)
    production_date = fake.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d')
    
    # Anotasyon bilgisi (xmin, ymin, width, height)
    shape_attributes = json.loads(row['region_shape_attributes'])   
    xmin = shape_attributes['x']
    ymin = shape_attributes['y']
    width = shape_attributes['width']
    height = shape_attributes['height']

    # SQL'e ürün bilgilerini ekleme
    cursor.execute('''
        INSERT INTO products (image_path, production_date, xmin, ymin, width, height)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (image_path, production_date, xmin, ymin, width, height))

# Ürünlerin kalite kontrol bilgilerini ekleme, quality_grade boş (NULL)
cursor.execute('SELECT product_id, image_path, xmin, ymin, width, height FROM products')
products = cursor.fetchall()

for product in products:
    product_id = product[0]
    image_path = product[1]
    xmin = product[2]
    ymin = product[3]
    width = product[4]
    height = product[5]
    
    cursor.execute('''
        INSERT INTO quality_control (product_id, image_path, qc_date, quality_grade, notes, xmin, ymin, width, height)
        VALUES (?, ?, CURRENT_DATE, NULL, NULL, ?, ?, ?, ?)
    ''', (product_id, image_path, xmin, ymin, width, height))

 
conn.commit()
conn.close()

print("Ürünler ve kalite kontrol bilgileri başarıyla veritabanına eklendi!")


Ürünler ve kalite kontrol bilgileri başarıyla veritabanına eklendi!


In [22]:
df

,Unnamed: 0,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,0,p1.JPG,167996,{},12,0,"{""name"":""rect"",""x"":70,""y"":46,""width"":120,""heig...","{""type"":""intact""}"
1,1,p1.JPG,167996,{},12,1,"{""name"":""rect"",""x"":195,""y"":60,""width"":118,""hei...","{""type"":""intact""}"
2,2,p1.JPG,167996,{},12,2,"{""name"":""rect"",""x"":318,""y"":25,""width"":144,""hei...","{""type"":""damaged""}"
3,3,p1.JPG,167996,{},12,3,"{""name"":""rect"",""x"":474,""y"":13,""width"":135,""hei...","{""type"":""intact""}"
4,4,p1.JPG,167996,{},12,4,"{""name"":""rect"",""x"":45,""y"":162,""width"":123,""hei...","{""type"":""intact""}"
...,...,...,...,...,...,...,...,...
1854,1854,t1.JPG,174414,{},6,1,"{""name"":""rect"",""x"":232,""y"":34,""width"":147,""hei...","{""type"":""intact""}"
1855,1855,t1.JPG,174414,{},6,2,"{""name"":""rect"",""x"":451,""y"":89,""width"":124,""hei...","{""type"":""intact""}"
1856,1856,t1.JPG,174414,{},6,3,"{""name"":""rect"",""x"":80,""y"":248,""width"":138,""hei...","{""type"":""damaged""}"
1857,1857,t1.JPG,174414,{},6,4,"{""name"":""rect"",""x"":244,""y"":284,""width"":146,""he...","{""type"":""intact""}"


In [23]:
conn = sqlite3.connect('jar_lids_db_v3.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT * FROM products")
products = cursor.fetchall()
print("Products Table:")
for product in products:
    print(product)
cursor.execute("SELECT * FROM quality_control")
quality_controls = cursor.fetchall()
print("\nQuality Control Table:")
for qc in quality_controls:
    print(qc)
conn.close()

Products Table:
(1, 'E:\\datasets\\defect_detection\\p1.JPG', '2024-07-05', 70, 46, 120, 112)
(2, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-07-04', 195, 60, 118, 99)
(3, 'E:\\datasets\\defect_detection\\p1.JPG', '2024-07-28', 318, 25, 144, 120)
(4, 'E:\\datasets\\defect_detection\\p1.JPG', '2022-12-10', 474, 13, 135, 122)
(5, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-09-29', 45, 162, 123, 116)
(6, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-06-09', 174, 166, 137, 130)
(7, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-07-07', 318, 146, 158, 148)
(8, 'E:\\datasets\\defect_detection\\p1.JPG', '2024-05-30', 473, 150, 138, 144)
(9, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-07-08', 51, 291, 135, 141)
(10, 'E:\\datasets\\defect_detection\\p1.JPG', '2024-04-24', 193, 314, 135, 138)
(11, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-03-15', 335, 309, 132, 132)
(12, 'E:\\datasets\\defect_detection\\p1.JPG', '2023-08-02', 481, 310, 134, 139)
(13, 'E:\\datasets\\defect_de

# sql_den dataframe geçişi 

In [24]:
db_path='jar_lids_db_v3.sqlite'
connection=sqlite3.connect(db_path)
cursor=connection.cursor()

In [25]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables=cursor.fetchall()

In [29]:
query="SELECT * FROM products"
df_products=pd.read_sql_query(query,connection)

In [30]:
df_products

,product_id,image_path,production_date,xmin,ymin,width,height
0,1,E:\datasets\defect_detection\p1.JPG,2024-07-05,70,46,120,112
1,2,E:\datasets\defect_detection\p1.JPG,2023-07-04,195,60,118,99
2,3,E:\datasets\defect_detection\p1.JPG,2024-07-28,318,25,144,120
3,4,E:\datasets\defect_detection\p1.JPG,2022-12-10,474,13,135,122
4,5,E:\datasets\defect_detection\p1.JPG,2023-09-29,45,162,123,116
...,...,...,...,...,...,...,...
1854,1855,E:\datasets\defect_detection\t1.JPG,2023-04-17,232,34,147,146
1855,1856,E:\datasets\defect_detection\t1.JPG,2024-04-22,451,89,124,126
1856,1857,E:\datasets\defect_detection\t1.JPG,2024-09-10,80,248,138,138
1857,1858,E:\datasets\defect_detection\t1.JPG,2024-03-07,244,284,146,150


In [31]:
query="SELECT * FROM quality_control"
df_quality=pd.read_sql_query(query,connection)

# kalite kontrol 

In [32]:
import pandas as pd 
from ultralytics import YOLO
import sqlite3
import torch

In [33]:
model_path="C:\\Users\\ninja\\Desktop\\Projeler\\produc_defect_detection\\dataset\\best_89.pt"

In [34]:
model = YOLO(model_path)
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [39]:
df_products

,product_id,image_path,production_date,xmin,ymin,width,height
0,1,E:\datasets\defect_detection\p1.JPG,2024-07-05,70,46,120,112
1,2,E:\datasets\defect_detection\p1.JPG,2023-07-04,195,60,118,99
2,3,E:\datasets\defect_detection\p1.JPG,2024-07-28,318,25,144,120
3,4,E:\datasets\defect_detection\p1.JPG,2022-12-10,474,13,135,122
4,5,E:\datasets\defect_detection\p1.JPG,2023-09-29,45,162,123,116
...,...,...,...,...,...,...,...
1854,1855,E:\datasets\defect_detection\t1.JPG,2023-04-17,232,34,147,146
1855,1856,E:\datasets\defect_detection\t1.JPG,2024-04-22,451,89,124,126
1856,1857,E:\datasets\defect_detection\t1.JPG,2024-09-10,80,248,138,138
1857,1858,E:\datasets\defect_detection\t1.JPG,2024-03-07,244,284,146,150


In [36]:
import os
import pandas as pd
from ultralytics import YOLO

 
base_image_path = 'E:\\datasets\\defect_detection'
results_list = []

for image_file in os.listdir(base_image_path):
    if image_file.endswith(('.JPG', '.png', '.jpeg')):  
        image_path = os.path.join(base_image_path, image_file)
        
        print(f'Processing image: {image_path}')

        
        results = model(source=image_path) 
        for result in results:
            boxes = result.boxes
            data = boxes.data

             
            bozuk_kavanozlar = []
            for i in range(data.shape[0]):
                if data[i, 5] == 1:  # Sınıf 1 bozuk kavanoz 
                    xmin, ymin, xmax, ymax = data[i, :4]
                    w = xmax - xmin
                    h = ymax - ymin
                    confidence = data[i, 4].item() 
                    bozuk_kavanozlar.append((xmin.item(), ymin.item(), w.item(), h.item(),confidence))

             
            if bozuk_kavanozlar:
                for bozuk_kavanoz in bozuk_kavanozlar:
                    results_list.append({
                        'image_path': image_path,
                        'xmin': bozuk_kavanoz[0],
                        'ymin': bozuk_kavanoz[1],
                        'width': bozuk_kavanoz[2],
                        'height': bozuk_kavanoz[3],
                        'confidence':bozuk_kavanoz[4]
                    })

 
results_df = pd.DataFrame(results_list)

 
results_df.to_csv('bozuk_kavanozlar.csv', index=False)
print("Sonuçlar 'bozuk_kavanozlar.csv' dosyasına yazıldı.")

Processing image: E:\datasets\defect_detection\p1.JPG

image 1/1 E:\datasets\defect_detection\p1.JPG: 480x640 10 intacts, 2 damageds, 24.9ms
Speed: 2.0ms preprocess, 24.9ms inference, 85.8ms postprocess per image at shape (1, 3, 480, 640)
Processing image: E:\datasets\defect_detection\p10.JPG

image 1/1 E:\datasets\defect_detection\p10.JPG: 480x640 3 intacts, 3 damageds, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Processing image: E:\datasets\defect_detection\p100.JPG

image 1/1 E:\datasets\defect_detection\p100.JPG: 480x640 3 intacts, 7 damageds, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Processing image: E:\datasets\defect_detection\p101.JPG

image 1/1 E:\datasets\defect_detection\p101.JPG: 480x640 4 intacts, 8 damageds, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
Processing image: E:\datasets\defect_detection

In [37]:
results_df

,image_path,xmin,ymin,width,height,confidence
0,E:\datasets\defect_detection\p1.JPG,480.116241,311.409485,134.564667,138.296631,0.960235
1,E:\datasets\defect_detection\p1.JPG,319.554321,26.222912,145.310059,119.544006,0.951984
2,E:\datasets\defect_detection\p10.JPG,41.807983,30.568069,201.027435,206.066040,0.978768
3,E:\datasets\defect_detection\p10.JPG,99.093292,247.639114,166.019501,167.729843,0.957409
4,E:\datasets\defect_detection\p10.JPG,271.651855,264.134155,176.401855,184.412598,0.929045
...,...,...,...,...,...,...
927,E:\datasets\defect_detection\t4.JPG,62.602219,59.180809,148.858459,162.027893,0.928838
928,E:\datasets\defect_detection\t4.JPG,463.215210,99.630905,128.190186,131.063995,0.902604
929,E:\datasets\defect_detection\t5.JPG,245.164673,91.103790,137.907959,134.447632,0.976160
930,E:\datasets\defect_detection\t5.JPG,248.318375,265.267212,157.463120,156.463501,0.948087


In [40]:
df_quality

,qc_id,product_id,image_path,qc_date,quality_grade,notes,xmin,ymin,width,height
0,1,1,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,70,46,120,112
1,2,2,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,195,60,118,99
2,3,3,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,318,25,144,120
3,4,4,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,474,13,135,122
4,5,5,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,45,162,123,116
...,...,...,...,...,...,...,...,...,...,...
1854,1855,1855,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,232,34,147,146
1855,1856,1856,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,451,89,124,126
1856,1857,1857,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,80,248,138,138
1857,1858,1858,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,244,284,146,150


In [ ]:
df

# tespitin quality_controle geçirilmesi 

In [41]:
def calculate_distance(x1, y1, w1, h1, x2, y2, w2, h2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (w1 - w2)**2 + (h1 - h2)**2)


df_quality['xmin'] = df_quality['xmin'].astype(float)
df_quality['ymin'] = df_quality['ymin'].astype(float)
df_quality['width'] = df_quality['width'].astype(float)
df_quality['height'] = df_quality['height'].astype(float)

for index_res, row_res in results_df.iterrows():
     
    image_path_res = row_res['image_path']
    xmin_res, ymin_res, width_res, height_res = row_res['xmin'], row_res['ymin'], row_res['width'], row_res['height']
    
    filtered_df = df_quality[df_quality['image_path'] == image_path_res]
    min_distance = float('inf')
    closest_row = None
    for index_df, row_df in filtered_df.iterrows():
        xmin_df, ymin_df, width_df, height_df = row_df['xmin'], row_df['ymin'], row_df['width'], row_df['height']
        distance = calculate_distance(xmin_df, ymin_df, width_df, height_df, xmin_res, ymin_res, width_res, height_res)
        
        if distance < min_distance:
            min_distance = distance
            closest_row = index_df

    if closest_row is not None:
        confidence = row_res['confidence']

        # confidence değerine göre en fazla zarar görmüş kavanoz kestirimi yapılabilir 
        if confidence > 0.94:
            quality_grade = "High_damage"
        elif confidence > 0.64:
            quality_grade = "Medium_damage"
        else:
            quality_grade = "Low_damage"

         
        notes = f"Confidence: {confidence:.2f}, Distance: {min_distance:.2f}"

         
        df_quality.loc[closest_row, 'quality_grade'] = quality_grade
        df_quality.loc[closest_row, 'notes'] = notes

 
df_quality.to_csv('updated_quality_control.csv', index=False)
print("Tablo başarıyla güncellendi ve kaydedildi.")


Tablo başarıyla güncellendi ve kaydedildi.


In [43]:
df_quality.to_excel("quality_control_data_v2.xlsx")

In [49]:
df_quality

,qc_id,product_id,image_path,qc_date,quality_grade,notes,xmin,ymin,width,height
0,1,1,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,70.0,46.0,120.0,112.0
1,2,2,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,195.0,60.0,118.0,99.0
2,3,3,E:\datasets\defect_detection\p1.JPG,2024-09-21,High_damage,"Confidence: 0.95, Distance: 2.42",318.0,25.0,144.0,120.0
3,4,4,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,474.0,13.0,135.0,122.0
4,5,5,E:\datasets\defect_detection\p1.JPG,2024-09-21,None,None,45.0,162.0,123.0,116.0
...,...,...,...,...,...,...,...,...,...,...
1854,1855,1855,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,232.0,34.0,147.0,146.0
1855,1856,1856,E:\datasets\defect_detection\t1.JPG,2024-09-21,Low_damage,"Confidence: 0.38, Distance: 1.24",451.0,89.0,124.0,126.0
1856,1857,1857,E:\datasets\defect_detection\t1.JPG,2024-09-21,High_damage,"Confidence: 0.97, Distance: 4.45",80.0,248.0,138.0,138.0
1857,1858,1858,E:\datasets\defect_detection\t1.JPG,2024-09-21,None,None,244.0,284.0,146.0,150.0


In [48]:
 
db_path = 'jar_lids_db_v3.sqlite'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS eski_quality_control AS SELECT * FROM quality_control")
connection.commit()

print("Eski tablo 'eski_quality_control' olarak yedeklendi.")

cursor.execute("DROP TABLE IF EXISTS quality_control")
connection.commit()

df_quality.to_sql('quality_control', connection, if_exists='replace', index=False)
print("Güncellenmiş DataFrame 'quality_control' tablosuna başarıyla yazıldı.")
connection.close()

Eski tablo 'eski_quality_control' olarak yedeklendi.
Güncellenmiş DataFrame 'quality_control' tablosuna başarıyla yazıldı.


In [1]:
df_quality

NameError: name 'df_quality' is not defined